# IMB Applied Data Science Capston Project
### Final Report

#### Opening a new Chinese food restaurant in Great Los Angeles Area 
1. scrap data about cities in Los Angeles metropolitan area
2. get geographic information for these cities
3. obtain the venue data from Foursquare API
4. K Means Cluster
5. select the best cluster to open a new Chinese restaurant

### import libraries

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### extract data from Wikipedia

In [97]:
data = requests.get("https://en.wikipedia.org/wiki/Los_Angeles_metropolitan_area#Urban_areas_of_the_region").text

In [98]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [101]:
bigla=[]
for row in soup.find_all("div", class_="mw-parser-output")[0].findAll("tr"):
    bigla.append(row.text)

In [121]:
#extract main cities in Los Angeles metropolitan area
cityli = []
cities = bigla[34].split('\n')
for city in cities:
    if city !='':
        c = city.split(' –')[0]
        cityli.append(c)

In [123]:
cityli

['Los Angeles',
 'Long Beach',
 'Anaheim',
 'Santa Ana',
 'Irvine',
 'Santa Clarita',
 'Glendale',
 'Huntington Beach',
 'Garden Grove',
 'Lancaster',
 'Palmdale',
 'Pomona',
 'Torrance',
 'Pasadena',
 'Fullerton',
 'Orange',
 'East Los Angeles',
 'El Monte',
 'Costa Mesa',
 'Downey',
 'Inglewood',
 'West Covina',
 'Norwalk',
 'Burbank',
 'Compton',
 'Mission Viejo',
 'South Gate',
 'Carson',
 'Santa Monica',
 'Westminster',
 'Hawthorne',
 'Whittier',
 'Lake Forest',
 'Newport Beach',
 'Alhambra',
 'Buena Park',
 'Lakewood',
 'Tustin',
 'Bellflower',
 'Baldwin Park',
 'Lynwood',
 'Yorba Linda',
 'Redondo Beach',
 'Laguna Niguel',
 'San Clemente',
 'Pico Rivera',
 'Montebello',
 'La Habra',
 'Monterey Park',
 'Gardena',
 'Arcadia',
 'Huntington Park',
 'Hacienda Heights',
 'Diamond Bar',
 'Fountain Valley',
 'Rosemead',
 'Paramount',
 'Glendora',
 'Placentia',
 'South Whittier']

In [124]:
len(cityli)

60

### create geographical information table

In [125]:
#get latitude and longitude of thoes districs
geodata =[]
for dist in cityli:
    dis = [dist]
    geo = geocoder.arcgis('{}, Los Angeles'.format(dist))
    geoinfo = geo.latlng
    info = dis+geoinfo
    geodata.append(info)

In [126]:
#created a dataframe for the list
ladf = pd.DataFrame(geodata, columns = ['Neighborhood', 'Latitude','Longitude'])
ladf.shape

(60, 3)

In [127]:
ladf.head()

,Neighborhood,Latitude,Longitude
0,Los Angeles,34.053610,-118.245500
1,Long Beach,33.766720,-118.192400
2,Anaheim,33.781556,-118.223196
3,Santa Ana,14.216670,-89.566670
4,Irvine,34.147946,-118.385580


### create a map of Los Anges with neighborhoods

In [128]:
geola = geocoder.arcgis('{}, Los Angeles'.format('Los Angeles'))
geoinfola = geola.latlng

In [129]:
# create map of Los Angeles using latitude and longitude values
latitude = geoinfola[0]
longitude=geoinfola[1]
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ladf['Latitude'], ladf['Longitude'], ladf['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

### use the Foursquare API to explore the neighborhoods

In [199]:

# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


Now, let's get the top 150 venues that are within a radius of 1500 meters.

In [151]:
ladf.head()

,Neighborhood,Latitude,Longitude
0,Los Angeles,34.053610,-118.245500
1,Long Beach,33.766720,-118.192400
2,Anaheim,33.781556,-118.223196
3,Santa Ana,14.216670,-89.566670
4,Irvine,34.147946,-118.385580


In [152]:
LIMIT = 150
radius = 1500
venues = []

for lat, long, neighborhood in zip(ladf['Latitude'], ladf['Longitude'], ladf['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    # get relevant information for each nearby venue
    for venue in results:
        venues.append((neighborhood,lat, long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [153]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4095, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Los Angeles,34.05361,-118.2455,Grand Park,34.055034,-118.245179,Park
1,Los Angeles,34.05361,-118.2455,Birds & Bees,34.052090,-118.247376,Speakeasy
2,Los Angeles,34.05361,-118.2455,Redbird,34.050666,-118.244068,American Restaurant
3,Los Angeles,34.05361,-118.2455,Badmaash,34.051342,-118.244571,Indian Restaurant
4,Los Angeles,34.05361,-118.2455,Walt Disney Concert Hall,34.055511,-118.249284,Concert Hall


In [155]:
#check number of venues for each neighborhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alhambra,65,65,65,65,65,65
Anaheim,18,18,18,18,18,18
Arcadia,100,100,100,100,100,100
Baldwin Park,62,62,62,62,62,62
Bellflower,55,55,55,55,55,55
Buena Park,72,72,72,72,72,72
Burbank,100,100,100,100,100,100
Carson,92,92,92,92,92,92
Compton,38,38,38,38,38,38


In [159]:
# get unique category
venues_df['VenueCategory'].unique()

array(['Park', 'Speakeasy', 'American Restaurant', 'Indian Restaurant',
       'Concert Hall', 'Art Museum', 'Building', 'Market', 'Coffee Shop',
       'Performing Arts Venue', 'Music Venue', 'Ice Cream Shop',
       'Filipino Restaurant', 'Plaza', 'Cheese Shop', 'School',
       'Breakfast Spot', 'Bookstore', 'Theater', 'Deli / Bodega',
       'Italian Restaurant', 'Latin American Restaurant', 'Jazz Club',
       'Opera House', 'Candy Store', 'Mexican Restaurant',
       'Seafood Restaurant', 'Thai Restaurant', 'Train Station', 'Bar',
       'Supermarket', 'Sushi Restaurant', 'Udon Restaurant',
       'Clothing Store', 'New American Restaurant', 'Bakery', 'Museum',
       'French Restaurant', 'Art Gallery', 'Historic Site',
       'Mediterranean Restaurant', 'High School', 'Gastropub',
       'BBQ Joint', 'Ramen Restaurant', 'Office', 'Stationery Store',
       'Restaurant', 'Taco Place', 'Japanese Restaurant', 'Hookah Bar',
       'Shopping Mall', 'Salad Place', 'Pizza Place',
     

In [160]:
# find all categroy related to Chinese food restaurant
#found there are four venue category related to restaurants that provide Chinese food 
res_list = ['Chinese Restaurant', 'Cantonese Restaurant', 'Dim Sum Restaurant','Szechuan Restaurant']

### apply one hot encoding for analyzing

In [183]:
# one hot encoding
la_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

print(la_onehot.shape)
la_onehot.head()

(4095, 320)


,Yoga Studio,ATM,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport Food Court,American Restaurant,Andhra Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Port,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,State / Provincial Park,Stationery Store,Steakhouse,Storage Fac

In [184]:
#group rows by neighborhood 
#by taking mean of the frequency of the occurance of each categore
la_grouped = la_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(la_grouped.shape)
la_grouped.head()

(60, 320)


,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport Food Court,American Restaurant,Andhra Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Port,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,State / Provincial Park,Stationery Store,Steakhouse,Storage Fac

### apply KNN to cluster neighborhoods

In [185]:
#create a dataframe that only contain chinese food restaurant information
la_ch_res = la_grouped[res_list+["Neighborhood"]]
la_ch_res.head()

,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Neighborhood
0,0.015385,0.0,0.0,0.00,Alhambra
1,0.000000,0.0,0.0,0.00,Anaheim
2,0.010000,0.0,0.0,0.01,Arcadia
3,0.032258,0.0,0.0,0.00,Baldwin Park
4,0.018182,0.0,0.0,0.00,Bellflower


In [186]:
#create a model with 4 clusters
kclusters = 4

la_clustering = la_ch_res.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 3, 0, 0, 0, 3, 0, 3, 3])

In [188]:
#attach the cluster number to df
la_ch_res['cluster'] = kmeans.labels_

C:\Users\Lin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [189]:
la_ch_res.head()

,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Neighborhood,cluster
0,0.015385,0.0,0.0,0.00,Alhambra,0
1,0.000000,0.0,0.0,0.00,Anaheim,3
2,0.010000,0.0,0.0,0.01,Arcadia,3
3,0.032258,0.0,0.0,0.00,Baldwin Park,0
4,0.018182,0.0,0.0,0.00,Bellflower,0


In [190]:
la_ch_res.groupby('cluster').count()

,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Neighborhood
cluster,,,,,
0,22,22,22,22,22
1,5,5,5,5,5
2,3,3,3,3,3
3,30,30,30,30,30


### visualize the resulting clusters on map

In [191]:
#get latitude and longitude from previous table
la_ch_res = la_ch_res.join(ladf.set_index("Neighborhood"), on="Neighborhood")

print(la_ch_res.shape)
la_ch_res.head()

(60, 8)


,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Neighborhood,cluster,Latitude,Longitude
0,0.015385,0.0,0.0,0.00,Alhambra,0,34.093700,-118.127270
1,0.000000,0.0,0.0,0.00,Anaheim,3,33.781556,-118.223196
2,0.010000,0.0,0.0,0.01,Arcadia,3,34.136350,-118.038670
3,0.032258,0.0,0.0,0.00,Baldwin Park,0,34.085360,-117.960350
4,0.018182,0.0,0.0,0.00,Bellflower,0,33.883260,-118.122290


In [193]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_ch_res['Latitude'], la_ch_res['Longitude'], la_ch_res['Neighborhood'], la_ch_res['cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### analysis each cluster

cluster 0

In [195]:
la_ch_res[la_ch_res['cluster'] == 0]

,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Neighborhood,cluster,Latitude,Longitude
0,0.015385,0.0,0.0,0.0,Alhambra,0,34.093700,-118.127270
3,0.032258,0.0,0.0,0.0,Baldwin Park,0,34.085360,-117.960350
4,0.018182,0.0,0.0,0.0,Bellflower,0,33.883260,-118.122290
5,0.027778,0.0,0.0,0.0,Buena Park,0,34.129020,-117.976760
7,0.032609,0.0,0.0,0.0,Carson,0,33.831610,-118.262090
10,0.029412,0.0,0.0,0.0,Diamond Bar,0,34.000860,-117.829070
17,0.030000,0.0,0.0,0.0,Gardena,0,33.883790,-118.307020
21,0.020619,0.0,0.0,0.0,Hawthorne,0,33.918220,-118.352730
25,0.020202,0.0,0.0,0.0,Irvine,0,34.147946,-118.385580
28,0.023810,0.0,0.0,0.0,Lake Forest,0,34.117727,-117.734320


cluster 1

In [196]:
la_ch_res[la_ch_res['cluster'] == 1]

,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Neighborhood,cluster,Latitude,Longitude
16,0.064516,0.00000,0.00000,0.00000,Garden Grove,1,34.242510,-118.529790
20,0.075000,0.00000,0.00000,0.00000,Hacienda Heights,1,33.994130,-117.972520
22,0.076087,0.01087,0.01087,0.01087,Huntington Beach,1,34.130765,-118.069415
38,0.054054,0.00000,0.00000,0.00000,Norwalk,1,33.916810,-118.072520
44,0.095238,0.00000,0.00000,0.00000,Placentia,1,33.994514,-117.953045


cluster 2

In [197]:
la_ch_res[la_ch_res['cluster'] == 2]

,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Neighborhood,cluster,Latitude,Longitude
15,0.130000,0.010000,0.010000,0.020000,Fullerton,2,33.985894,-117.906814
36,0.184615,0.000000,0.030769,0.015385,Monterey Park,2,34.059790,-118.126510
47,0.117647,0.014706,0.000000,0.014706,Rosemead,2,34.080750,-118.075620


cluster 3

In [198]:
la_ch_res[la_ch_res['cluster'] == 3]

,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant,Szechuan Restaurant,Neighborhood,cluster,Latitude,Longitude
1,0.000000,0.0,0.000000,0.00,Anaheim,3,33.781556,-118.223196
2,0.010000,0.0,0.000000,0.01,Arcadia,3,34.136350,-118.038670
6,0.010000,0.0,0.000000,0.00,Burbank,3,34.181820,-118.307760
8,0.000000,0.0,0.000000,0.00,Compton,3,33.895690,-118.224360
9,0.000000,0.0,0.000000,0.00,Costa Mesa,3,33.900695,-117.996570
11,0.010101,0.0,0.000000,0.00,Downey,3,33.940770,-118.127680
12,0.000000,0.0,0.000000,0.00,East Los Angeles,3,34.033470,-118.159090
13,0.000000,0.0,0.014286,0.00,El Monte,3,34.071050,-118.030940
14,0.000000,0.0,0.000000,0.00,Fountain Valley,3,34.095721,-118.276428
18,0.000000,0.0,0.000000,0.00,Glendale,3,34.146330,-118.248640
